In [7]:
import pandas as pd
import numpy as np
import duckdb

In [12]:
df_hc = pd.read_csv('..\..\Go_Daddy_Data\Data\Data\RDC_Inventory_Core_Metrics_County_History.csv') #Done
df_bls = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\bls-emp_by_cfips.xlsx') #Done
df_ctr = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Corporate_tax_rate.csv') #Done
df_rm_FY17 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2017_50_rev.xlsx') #Done
df_rm_FY18 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2018_50_County_rev.xlsx') #Done
df_rm_FY19 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2019_50_County_rev.xlsx') #Done
df_rm_FY20 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2020_50_County_rev.xlsx') #Done
df_rm_FY21 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2021_50_County.xlsx') #Done
df_rm_FY22 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2022_FMR_50_county_rev.xlsx') #Done
df_rm_FY23 = pd.read_excel(r'..\..\Go_Daddy_Data\Data\Data\FY2023_FMR_50_county.xlsx') #Done
df_me = pd.read_csv(r'..\..\Go_Daddy_Data\Data\Data\Medicare_Monthly_Enrollment_September_2022.csv') #Done


df_cs = pd.read_csv('..\Data\data_zip\census_starter.csv') #Done
df_test = pd.read_csv(r'..\Data\data_zip\test.csv')
df_train = pd.read_csv(r'..\Data\data_zip\train.csv')

In [13]:
df_train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [14]:
fips_county_state = duckdb.query("SELECT distinct cfips, county, state FROM df_train").to_df()
fips_county_state

,cfips,county,state
0,48213,Henderson County,Texas
1,48215,Hidalgo County,Texas
2,48217,Hill County,Texas
3,48219,Hockley County,Texas
4,48221,Hood County,Texas
...,...,...,...
3130,41023,Grant County,Oregon
3131,39115,Morgan County,Ohio
3132,46105,Perkins County,South Dakota
3133,42085,Mercer County,Pennsylvania


In [15]:
df_test['first_day_of_month'] = pd.to_datetime(df_test['first_day_of_month'])
df_test['year'] = df_test['first_day_of_month'].dt.year

In [16]:
df_test_comb = df_test.merge(fips_county_state, how='left', left_on=['cfips'], right_on=['cfips']
                               , suffixes=['_orig', '_bls'])
df_test_comb

,row_id,cfips,first_day_of_month,year,county,state
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama
...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming


In [17]:
df_bls2 = df_bls[df_bls['period'] != 'M13']
df_bls2_2 = pd.pivot_table(df_bls2, values='value', index=['cfips', 'first_day_of_month'], columns=['measure_type'], aggfunc=np.sum).reset_index()
df_bls2_2['first_day_of_month'] = pd.to_datetime(df_bls2_2['first_day_of_month'])
df_bls2_2['first_day_of_month_lag_12'] = df_bls2_2['first_day_of_month'] + pd.DateOffset(months=12)
#df_bls2_2 = df_bls2_2.drop(columns=['Labor force'])
df_bls2_2
df_test_comb = df_test_comb.merge(df_bls2_2, how='left', left_on=['cfips', 'first_day_of_month'], right_on=['cfips','first_day_of_month_lag_12']
                               , suffixes=['_orig', '_bls'])
df_test_comb

,row_id,cfips,first_day_of_month_orig,year,county,state,first_day_of_month_bls,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag_12
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,2021-11-01,25898.0,26474.0,576.0,2.2,2022-11-01
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,2021-11-01,97017.0,99271.0,2254.0,2.3,2022-11-01
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,2021-11-01,7741.0,8081.0,340.0,4.2,2022-11-01
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,2021-11-01,8416.0,8638.0,222.0,2.6,2022-11-01
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,2021-11-01,25016.0,25500.0,484.0,1.9,2022-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT


In [18]:
df_cs2 = pd.melt(df_cs, id_vars=['cfips'])
df_cs2['Year'] = df_cs2['variable'].str[-4:]
df_cs2['variable'] = df_cs2['variable'].str[:-5]
df_cs2.sort_values(['cfips']).reset_index(drop=True)
df_cs2 = pd.pivot_table(df_cs2, values='value', index=['cfips', 'Year'], columns=['variable'], aggfunc=np.sum).reset_index()
df_cs2['year_lag'] = (pd.to_datetime(df_cs2['Year']) + pd.DateOffset(years=2)).dt.year
#df_cs2 = df_cs2.drop(columns=['median_hh_inc'])
df_test_comb = df_test_comb.merge(df_cs2, how='left', left_on=['cfips', 'year'], right_on=['cfips','year_lag'], 
                              suffixes=['_orig', '_cps'])
df_test_comb

,row_id,cfips,first_day_of_month_orig,year,county,state,first_day_of_month_bls,Employment,Labor force,Unemployment,Unemployment rate,first_day_of_month_lag_12,Year,median_hh_inc,pct_bb,pct_college,pct_foreign_born,pct_it_workers,year_lag
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,2021-11-01,25898.0,26474.0,576.0,2.2,2022-11-01,2020,57982.0,82.7,16.7,2.3,0.6,2022
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,2021-11-01,97017.0,99271.0,2254.0,2.3,2022-11-01,2020,61756.0,85.1,20.2,3.4,1.0,2022
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,2021-11-01,7741.0,8081.0,340.0,4.2,2022-11-01,2020,34990.0,64.6,7.3,2.6,1.1,2022
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,2021-11-01,8416.0,8638.0,222.0,2.6,2022-11-01,2020,51721.0,76.1,7.4,1.6,1.7,2022
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,2021-11-01,25016.0,25500.0,484.0,1.9,2022-11-01,2020,48922.0,79.6,8.9,4.4,1.1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT,2021,76668.0,88.4,12.4,5.5,1.0,2023
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT,2021,94498.0,90.5,38.3,11.1,2.0,2023
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT,2021,75106.0,90.6,12.3,2.9,0.9,2023
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaT,NaN,NaN,NaN,NaN,NaT,2021,62271.0,85.4,17.2,1.0,1.1,2023


In [19]:
df_hc['date'] = pd.to_datetime(df_hc['month_date_yyyymm'].apply(str).str[0:4]+'-'\
                               +df_hc['month_date_yyyymm'].apply(str).str[4:6]+'-01')
keep = ['date', 'county_fips', 'active_listing_count', 'active_listing_count_yy', 'median_days_on_market', 
        'new_listing_count', 'price_reduced_count']
df_hc_vars = df_hc[keep]
df_hc_vars['date_lag_12'] = df_hc_vars['date'] + pd.DateOffset(months=12)
df_test_comb = df_test_comb.merge(df_hc_vars, how='left', left_on=['cfips', 'first_day_of_month_orig'], \
                                    right_on=['county_fips','date_lag_12'], suffixes=['_orig', '_hc'])
df_test_comb

C:\Users\jacks\AppData\Local\Temp\ipykernel_15688\3840422679.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hc_vars['date_lag_12'] = df_hc_vars['date'] + pd.DateOffset(months=12)


,row_id,cfips,first_day_of_month_orig,year,county,state,first_day_of_month_bls,Employment,Labor force,Unemployment,...,pct_it_workers,year_lag,date,county_fips,active_listing_count,active_listing_count_yy,median_days_on_market,new_listing_count,price_reduced_count,date_lag_12
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,2021-11-01,25898.0,26474.0,576.0,...,0.6,2022,2021-11-01,1001.0,55.0,-0.4086,40.0,76.0,8.0,2022-11-01
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,2021-11-01,97017.0,99271.0,2254.0,...,1.0,2022,2021-11-01,1003.0,1233.0,-0.2073,55.0,794.0,336.0,2022-11-01
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,2021-11-01,7741.0,8081.0,340.0,...,1.1,2022,2021-11-01,1005.0,49.0,-0.5847,98.0,18.0,10.0,2022-11-01
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,2021-11-01,8416.0,8638.0,222.0,...,1.7,2022,2021-11-01,1007.0,9.0,-0.5714,110.0,6.0,0.0,2022-11-01
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,2021-11-01,25016.0,25500.0,484.0,...,1.1,2022,2021-11-01,1009.0,76.0,-0.0793,73.0,52.0,22.0,2022-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaT,NaN,NaN,NaN,...,1.0,2023,2022-06-01,56037.0,69.0,-0.3010,36.0,54.0,14.0,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaT,NaN,NaN,NaN,...,2.0,2023,2022-06-01,56039.0,103.0,0.1389,35.0,58.0,16.0,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaT,NaN,NaN,NaN,...,0.9,2023,2022-06-01,56041.0,49.0,0.0889,40.0,22.0,16.0,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaT,NaN,NaN,NaN,...,1.1,2023,2022-06-01,56043.0,26.0,-0.1034,31.0,10.0,12.0,2023-06-01


In [20]:
df_me_yearly = df_me[(df_me['MONTH'] == 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)

df_me_monthly = df_me[(df_me['MONTH'] != 'Year') & (df_me['BENE_GEO_LVL'] == 'County')]
df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)

df_me_monthly['date'] = pd.to_datetime(df_me_monthly['YEAR'].astype(str)  + df_me_monthly['MONTH'], format='%Y%B')

C:\Users\jacks\AppData\Local\Temp\ipykernel_15688\2785904398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_yearly.BENE_FIPS_CD = df_me_yearly.BENE_FIPS_CD.astype(int)
C:\Users\jacks\AppData\Local\Temp\ipykernel_15688\2785904398.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_monthly.BENE_FIPS_CD = df_me_monthly.BENE_FIPS_CD.astype(int)
C:\Users\jacks\AppData\Local\Temp\ipykernel_15688\2785904398.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [21]:
date_df = pd.DataFrame(pd.date_range(start='2017-01-01', end='2021-08-01'), columns=['date'])

date_df['year'] = date_df.date.dt.year
date_df['day'] = date_df.date.dt.day
date_df = date_df[date_df.day == 1]

df_me_yearly_month = df_me_yearly.merge(date_df, how="left", left_on='YEAR', right_on='year')
df_me_yearly_month = df_me_yearly_month.drop(['year', 'day'], axis=1)
df_me_yearly_month

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,...,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,date
0,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-01-01
1,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-02-01
2,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-03-01
3,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-04-01
4,2017,Year,County,AL,Alabama,Autauga,1001,10510,5784,4725,...,2463,56,2407,9810,5085,4725,7009,2396,4613,2017-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183423,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-04-01
183424,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-05-01
183425,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-06-01
183426,2021,Year,County,VI,Virgin Islands,Unknown,78999,*,*,*,...,*,*,*,*,*,*,*,*,*,2021-07-01


In [22]:
df_me_final = pd.concat([df_me_monthly, df_me_yearly_month])
df_me_final = df_me_final.replace({'*': np.nan})
df_me_final['date_lag_12'] = df_me_final['date'] + pd.DateOffset(months=12)
keep = ['BENE_FIPS_CD','date_lag_12', 'TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
df_me_final_keep = df_me_final[keep]
me_values = ['TOT_BENES', 'AGED_TOT_BENES', 'DSBLD_TOT_BENES',
       'A_B_TOT_BENES', 'PRSCRPTN_DRUG_TOT_BENES']
for i in me_values:
    df_me_final_keep[i] = df_me_final_keep[i].astype(float)

#df_me_final_keep_dup = df_me_final_keep[df_me_final_keep[['BENE_FIPS_CD','date_lag_12']].duplicated()]
print(len(df_train))
df_test_comb = df_test_comb.merge(df_me_final_keep, how='left', left_on=['cfips', 'first_day_of_month_orig'], right_on=['BENE_FIPS_CD','date_lag_12'], 
                              suffixes=['_orig', '_me'])
df_test_comb

122265


C:\Users\jacks\AppData\Local\Temp\ipykernel_15688\4217313890.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_me_final_keep[i] = df_me_final_keep[i].astype(float)


,row_id,cfips,first_day_of_month_orig,year,county,state,first_day_of_month_bls,Employment,Labor force,Unemployment,...,new_listing_count,price_reduced_count,date_lag_12_orig,BENE_FIPS_CD,date_lag_12_me,TOT_BENES,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,2021-11-01,25898.0,26474.0,576.0,...,76.0,8.0,2022-11-01,1001,2022-11-01,11449.0,9059.0,2390.0,10582.0,7861.0
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,2021-11-01,97017.0,99271.0,2254.0,...,794.0,336.0,2022-11-01,1003,2022-11-01,57835.0,50778.0,7057.0,54120.0,43596.0
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,2021-11-01,7741.0,8081.0,340.0,...,18.0,10.0,2022-11-01,1005,2022-11-01,6212.0,4816.0,1396.0,5872.0,4958.0
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,2021-11-01,8416.0,8638.0,222.0,...,6.0,0.0,2022-11-01,1007,2022-11-01,4682.0,3470.0,1212.0,4387.0,3604.0
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,2021-11-01,25016.0,25500.0,484.0,...,52.0,22.0,2022-11-01,1009,2022-11-01,13229.0,10680.0,2549.0,12432.0,10648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaT,NaN,NaN,NaN,...,54.0,14.0,2023-06-01,56037,2023-06-01,6990.0,6060.0,930.0,6473.0,4894.0
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaT,NaN,NaN,NaN,...,58.0,16.0,2023-06-01,56039,2023-06-01,4018.0,3906.0,112.0,3701.0,2906.0
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaT,NaN,NaN,NaN,...,22.0,16.0,2023-06-01,56041,2023-06-01,3990.0,3458.0,532.0,3657.0,2865.0
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaT,NaN,NaN,NaN,...,10.0,12.0,2023-06-01,56043,2023-06-01,2054.0,1886.0,168.0,1879.0,1345.0


In [23]:
df_rm_FY17['fips'] = df_rm_FY17['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY17['year'] = 2017
df_rm_FY17 = df_rm_FY17.rename(columns={'Rent50_0': 'rent50_0', 'Rent50_1': 'rent50_1', 'Rent50_2': 'rent50_2', 'Rent50_3': 'rent50_3', 'Rent50_4': 'rent50_4'})
df_rm_FY17_keep = df_rm_FY17[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY18['fips'] = df_rm_FY18['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY18['year'] = 2018
df_rm_FY18_keep = df_rm_FY18[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY19['fips'] = df_rm_FY19['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY19['year'] = 2019
df_rm_FY19_keep = df_rm_FY19[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY20['fips'] = df_rm_FY20['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY20['year'] = 2020
df_rm_FY20_keep = df_rm_FY20[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY21['fips'] = df_rm_FY21['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY21['year'] = 2021
df_rm_FY21_keep = df_rm_FY21[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY22['fips'] = df_rm_FY22['state_code'].astype(str) + df_rm_FY22['county_code'].astype(str).str.zfill(3)
df_rm_FY22['fips'] = df_rm_FY22['fips'].astype(int)
df_rm_FY22['year'] = 2022
df_rm_FY22 = df_rm_FY22.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY22_keep = df_rm_FY22[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm_FY23['fips'] = df_rm_FY23['fips2010'].astype(str).str[:-5].astype(int)
df_rm_FY23['year'] = 2023
df_rm_FY23 = df_rm_FY23.rename(columns={'rent_50_0': 'rent50_0', 'rent_50_1': 'rent50_1', 'rent_50_2': 'rent50_2', 'rent_50_3': 'rent50_3', 'rent_50_4': 'rent50_4'})
df_rm_FY23_keep = df_rm_FY23[['fips', 'year', 'rent50_0', 'rent50_1', 'rent50_2', 'rent50_3', 'rent50_4']]

df_rm = pd.concat([df_rm_FY17_keep, df_rm_FY18_keep, df_rm_FY19_keep, df_rm_FY20_keep, df_rm_FY21_keep, df_rm_FY22_keep, df_rm_FY23_keep])

print(len(df_train))

print(len(df_rm))
df_rm_avg = df_rm.groupby(['fips', 'year']).mean().reset_index()

df_test_comb = df_test_comb.merge(df_rm_avg, how='left', left_on=['cfips', 'year'], right_on=['fips','year'], 
                              suffixes=['_orig', 'rm'])

df_test_comb

122265
33369


,row_id,cfips,first_day_of_month_orig,year,county,state,first_day_of_month_bls,Employment,Labor force,Unemployment,...,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,fips,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,2021-11-01,25898.0,26474.0,576.0,...,9059.0,2390.0,10582.0,7861.0,1001,687.0,816.0,976.0,1234.0,1596.0
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,2021-11-01,97017.0,99271.0,2254.0,...,50778.0,7057.0,54120.0,43596.0,1003,872.0,878.0,1149.0,1523.0,1938.0
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,2021-11-01,7741.0,8081.0,340.0,...,4816.0,1396.0,5872.0,4958.0,1005,558.0,562.0,739.0,914.0,1027.0
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,2021-11-01,8416.0,8638.0,222.0,...,3470.0,1212.0,4387.0,3604.0,1007,834.0,895.0,1029.0,1331.0,1436.0
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,2021-11-01,25016.0,25500.0,484.0,...,10680.0,2549.0,12432.0,10648.0,1009,834.0,895.0,1029.0,1331.0,1436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaT,NaN,NaN,NaN,...,6060.0,930.0,6473.0,4894.0,56037,834.0,853.0,963.0,1315.0,1640.0
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaT,NaN,NaN,NaN,...,3906.0,112.0,3701.0,2906.0,56039,1295.0,1303.0,1644.0,2316.0,2800.0
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaT,NaN,NaN,NaN,...,3458.0,532.0,3657.0,2865.0,56041,662.0,666.0,877.0,1208.0,1465.0
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaT,NaN,NaN,NaN,...,1886.0,168.0,1879.0,1345.0,56043,663.0,668.0,877.0,1246.0,1331.0


In [24]:
df_ctr2 = pd.melt(df_ctr, id_vars=['State'])
df_ctr2['value'] = df_ctr2['value'].str.rstrip('%').astype('float') / 100.0
df_ctr2['year_lag_1'] = (pd.to_datetime(df_ctr2['variable']) + pd.DateOffset(years=1)).dt.year
df_ctr2['State'] = df_ctr2['State'].str[:-1]
df_test_comb = df_test_comb.merge(df_ctr2, how='left', left_on=['state', 'year'], right_on=['State','year_lag_1'], 
                              suffixes=['_orig', 'ctr'])
df_test_comb

,row_id,cfips,first_day_of_month_orig,year,county,state,first_day_of_month_bls,Employment,Labor force,Unemployment,...,fips,rent50_0,rent50_1,rent50_2,rent50_3,rent50_4,State,variable,value,year_lag_1
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,2021-11-01,25898.0,26474.0,576.0,...,1001,687.0,816.0,976.0,1234.0,1596.0,Alabama,2021,0.0605,2022.0
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,2021-11-01,97017.0,99271.0,2254.0,...,1003,872.0,878.0,1149.0,1523.0,1938.0,Alabama,2021,0.0605,2022.0
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,2021-11-01,7741.0,8081.0,340.0,...,1005,558.0,562.0,739.0,914.0,1027.0,Alabama,2021,0.0605,2022.0
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,2021-11-01,8416.0,8638.0,222.0,...,1007,834.0,895.0,1029.0,1331.0,1436.0,Alabama,2021,0.0605,2022.0
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,2021-11-01,25016.0,25500.0,484.0,...,1009,834.0,895.0,1029.0,1331.0,1436.0,Alabama,2021,0.0605,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaT,NaN,NaN,NaN,...,56037,834.0,853.0,963.0,1315.0,1640.0,Wyoming,2022,0.0000,2023.0
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaT,NaN,NaN,NaN,...,56039,1295.0,1303.0,1644.0,2316.0,2800.0,Wyoming,2022,0.0000,2023.0
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaT,NaN,NaN,NaN,...,56041,662.0,666.0,877.0,1208.0,1465.0,Wyoming,2022,0.0000,2023.0
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaT,NaN,NaN,NaN,...,56043,663.0,668.0,877.0,1246.0,1331.0,Wyoming,2022,0.0000,2023.0


In [25]:
df_test_comb.columns

Index(['row_id', 'cfips', 'first_day_of_month_orig', 'year', 'county', 'state',
       'first_day_of_month_bls', 'Employment', 'Labor force', 'Unemployment',
       'Unemployment rate', 'first_day_of_month_lag_12', 'Year',
       'median_hh_inc', 'pct_bb', 'pct_college', 'pct_foreign_born',
       'pct_it_workers', 'year_lag', 'date', 'county_fips',
       'active_listing_count', 'active_listing_count_yy',
       'median_days_on_market', 'new_listing_count', 'price_reduced_count',
       'date_lag_12_orig', 'BENE_FIPS_CD', 'date_lag_12_me', 'TOT_BENES',
       'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'fips', 'rent50_0', 'rent50_1', 'rent50_2',
       'rent50_3', 'rent50_4', 'State', 'variable', 'value', 'year_lag_1'],
      dtype='object')

In [27]:
#drop median_days_on_market, rent50_0, Unemployment, Employment, Unemployment rate, active_listing_count_yy, TOT_BENES
df_test_final = df_test_comb[['row_id', 'cfips', 'first_day_of_month_orig', 'year', 'county', 'state',
       'Employment', 'Labor force',
       'median_hh_inc', 'pct_bb', 'pct_college', 'pct_foreign_born',
       'pct_it_workers',
       'active_listing_count',
       'new_listing_count', 'price_reduced_count',
       'AGED_TOT_BENES', 'DSBLD_TOT_BENES', 'A_B_TOT_BENES',
       'PRSCRPTN_DRUG_TOT_BENES', 'rent50_1', 'rent50_2',
       'rent50_3', 'rent50_4', 'value']]
df_test_final

,row_id,cfips,first_day_of_month_orig,year,county,state,Employment,Labor force,median_hh_inc,pct_bb,...,price_reduced_count,AGED_TOT_BENES,DSBLD_TOT_BENES,A_B_TOT_BENES,PRSCRPTN_DRUG_TOT_BENES,rent50_1,rent50_2,rent50_3,rent50_4,value
0,1001_2022-11-01,1001,2022-11-01,2022,Autauga County,Alabama,25898.0,26474.0,57982.0,82.7,...,8.0,9059.0,2390.0,10582.0,7861.0,816.0,976.0,1234.0,1596.0,0.0605
1,1003_2022-11-01,1003,2022-11-01,2022,Baldwin County,Alabama,97017.0,99271.0,61756.0,85.1,...,336.0,50778.0,7057.0,54120.0,43596.0,878.0,1149.0,1523.0,1938.0,0.0605
2,1005_2022-11-01,1005,2022-11-01,2022,Barbour County,Alabama,7741.0,8081.0,34990.0,64.6,...,10.0,4816.0,1396.0,5872.0,4958.0,562.0,739.0,914.0,1027.0,0.0605
3,1007_2022-11-01,1007,2022-11-01,2022,Bibb County,Alabama,8416.0,8638.0,51721.0,76.1,...,0.0,3470.0,1212.0,4387.0,3604.0,895.0,1029.0,1331.0,1436.0,0.0605
4,1009_2022-11-01,1009,2022-11-01,2022,Blount County,Alabama,25016.0,25500.0,48922.0,79.6,...,22.0,10680.0,2549.0,12432.0,10648.0,895.0,1029.0,1331.0,1436.0,0.0605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,2023,Sweetwater County,Wyoming,NaN,NaN,76668.0,88.4,...,14.0,6060.0,930.0,6473.0,4894.0,853.0,963.0,1315.0,1640.0,0.0000
25076,56039_2023-06-01,56039,2023-06-01,2023,Teton County,Wyoming,NaN,NaN,94498.0,90.5,...,16.0,3906.0,112.0,3701.0,2906.0,1303.0,1644.0,2316.0,2800.0,0.0000
25077,56041_2023-06-01,56041,2023-06-01,2023,Uinta County,Wyoming,NaN,NaN,75106.0,90.6,...,16.0,3458.0,532.0,3657.0,2865.0,666.0,877.0,1208.0,1465.0,0.0000
25078,56043_2023-06-01,56043,2023-06-01,2023,Washakie County,Wyoming,NaN,NaN,62271.0,85.4,...,12.0,1886.0,168.0,1879.0,1345.0,668.0,877.0,1246.0,1331.0,0.0000


In [28]:
df_test_final.to_csv('testing.csv')